In [4]:
import tensorflow.keras as K
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'  # suppress CPU msg

class MyLogger(K.callbacks.Callback):
    def __init__(self, n):
        self.n = n   # print loss & acc every n epochs

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.n == 0:
            curr_loss =logs.get('loss')
            curr_acc = logs.get('accuracy') * 100
            print("epoch = %4d  loss = %0.6f  acc = %0.2f%%" % \
            (epoch, curr_loss, curr_acc))

def main():
    print("\nBlood transfusion dataset example \n")
    np.random.seed(1)

  # 1. load data
  # print("Loading data into memory ")
    train_file = "C:/Users/lenovo/Desktop/IA/transfusion_train.data"
    test_file = "C:/Users/lenovo/Desktop/IA/transfusion_test.data"

    train_x_nenorm = np.loadtxt(train_file, delimiter=',', usecols=[0,1,2,3], dtype=np.float32)
    col0_norm = (train_x_nenorm[:,0]-min(train_x_nenorm[:,0])) / (max(train_x_nenorm[:,0])-min(train_x_nenorm[:,0]))
    col1_norm = (train_x_nenorm[:,1]-min(train_x_nenorm[:,1])) / (max(train_x_nenorm[:,1])-min(train_x_nenorm[:,1]))
    col2_norm = (train_x_nenorm[:,2]-min(train_x_nenorm[:,2])) / (max(train_x_nenorm[:,2])-min(train_x_nenorm[:,2]))
    col3_norm = (train_x_nenorm[:,3]-min(train_x_nenorm[:,3])) / (max(train_x_nenorm[:,3])-min(train_x_nenorm[:,3]))
    train_x=np.stack((col0_norm,col1_norm,col2_norm,col3_norm),axis=1)
    train_y = np.loadtxt(train_file, delimiter=',',
    usecols=[4], dtype=np.float32)
    test_x_nenorm = np.loadtxt(test_file, delimiter=',', 
    usecols=[0,1,2,3], dtype=np.float32)
    col0_norm1 = (test_x_nenorm[:,0]-min(test_x_nenorm[:,0])) / (max(test_x_nenorm[:,0])-min(test_x_nenorm[:,0]))
    col1_norm1 = (test_x_nenorm[:,1]-min(test_x_nenorm[:,1])) / (max(test_x_nenorm[:,1])-min(test_x_nenorm[:,1]))
    col2_norm1 = (test_x_nenorm[:,2]-min(test_x_nenorm[:,2])) / (max(test_x_nenorm[:,2])-min(test_x_nenorm[:,2]))
    col3_norm1 = (test_x_nenorm[:,3]-min(test_x_nenorm[:,3])) / (max(test_x_nenorm[:,3])-min(test_x_nenorm[:,3]))
    test_x=np.stack((col0_norm1,col1_norm1,col2_norm1,col3_norm1),axis=1)
    test_y =np.loadtxt(test_file, delimiter=',',
    usecols=[4], dtype=np.float32)


  # 2. define 4-(x-x)-1 deep NN model
    print("Creating 4-(8-8)-1 binary NN classifier \n")
    my_init = K.initializers.glorot_uniform(seed=1)
    model = K.models.Sequential()
    model.add(K.layers.Dense(units=8, input_dim=4,
    activation='tanh', kernel_initializer=my_init)) 
    model.add(K.layers.Dense(units=8, activation='tanh',
    kernel_initializer=my_init)) 
    model.add(K.layers.Dense(units=1, activation='sigmoid',
    kernel_initializer=my_init)) 

  # 3. compile model
    simple_sgd = K.optimizers.SGD(lr=0.01)  
    model.compile(loss='binary_crossentropy',
    optimizer=simple_sgd, metrics=['accuracy'])  

  # 4. train model
    max_epochs = 500
    my_logger = MyLogger(n=50)
    h = model.fit(train_x, train_y, batch_size=32,
    epochs=max_epochs, verbose=0, callbacks=[my_logger]) 

  # 5. evaluate model
    np.set_printoptions(precision=4, suppress=True)
    eval_results = model.evaluate(test_x, test_y, verbose=0) 
    print("\nLoss, accuracy on test data: ")
    print("%0.4f %0.2f%%" % (eval_results[0], \
    eval_results[1]*100))

  # 6. save model
  # mp = ".\\Models\\banknote_model.h5"
  # model.save(mp)

  # 7. make a prediction
    inpts = np.array([[0.5, 0.5, 0.5, 0.5]], dtype=np.float32)
    pred = model.predict(inpts)
    print("\nPredicting donator for: ")
    print(inpts)
    print("Probability that class = 1 (he/she donated blood):")
    print(pred)

if __name__=="__main__":
    main()


Blood transfusion dataset example 

Creating 4-(8-8)-1 binary NN classifier 



C:\Users\lenovo\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


epoch =    0  loss = 0.700298  acc = 48.61%
epoch =   50  loss = 0.481391  acc = 80.40%
epoch =  100  loss = 0.470273  acc = 80.56%
epoch =  150  loss = 0.459008  acc = 80.71%
epoch =  200  loss = 0.450142  acc = 81.48%
epoch =  250  loss = 0.444978  acc = 81.17%
epoch =  300  loss = 0.442533  acc = 81.02%
epoch =  350  loss = 0.441042  acc = 80.71%
epoch =  400  loss = 0.440452  acc = 80.71%
epoch =  450  loss = 0.440223  acc = 80.86%

Loss, accuracy on test data: 
0.9070 58.00%

Predicting donator for: 
[[0.5 0.5 0.5 0.5]]
Probability that class = 1 (he/she donated blood):
[[0.1529]]
